In [37]:
import paramiko
import getpass

host = "158.251.88.197"
port = 22
username = "iaraya"
password = getpass.getpass()

ssh = paramiko.SSHClient()
ssh.set_missing_host_key_policy(paramiko.AutoAddPolicy())
ssh.connect(host, port, username, password)

In [38]:
import random

def write_instance(L,W,H,boxes,filename) :
    txt = "1\n1 0\n"+str(L)+" "+str(W)+" "+str(H)+"\n";
    txt += str(len(boxes)) + "\n"
    for box in boxes:
        txt += str(box.id) + ' ' + str(box.l) + ' ' + str(box.rotx) \
            + ' ' + str(box.w) + ' ' + str(box.roty) \
            + ' ' + str(box.h) + ' ' + str(box.rotz) + ' ' + str(boxes[box])  + '\n'
    txt += '\n'

    text_file = open(filename, "w")
    text_file.write(txt)
    text_file.close()


In [39]:
import os
import json
from scp import SCPClient

def bsg_solve(L,W,H,boxes,id2box, time=1):
    filename="tmp_instance_"+ str(random.randint(10000, 99999))
    
    write_instance(L,W,H,boxes,filename)
    
    scp = SCPClient(ssh.get_transport())
    scp.put(filename, "/home/iaraya/clp/"+filename)

    stdin, stdout, stderr = ssh.exec_command("/home/iaraya/clp/BSG_CLP /home/iaraya/clp/"+filename+" -i 0 -t "+str(time)+" --json")
    lines = stdout.readlines()
    print(lines[-1])
    json_data = json.loads(lines[-1])
    loaded={}; remaining={}
    for item in json_data["loaded"]:
        loaded[id2box[item[0]]]=item[1]
    for item in json_data["remaining"]:
        remaining[id2box[item[0]]]=item[1]
        
    ssh.exec_command("rm /home/iaraya/clp/"+filename)
    os.remove(filename)
    return remaining, loaded, json_data["utilization"]


In [40]:
from box import box
import random
# id l   w g  rotx roty rotz nn
# 1	 108 0 76 0	   30	1	 7

def load_BRinstance(filename, inst=1, nbox = 1):
    boxes={}; id2box={}
    with open(filename) as f:
        n = int(next(f))
        for i in range(n):
            next(f)
            L, W, H =  [int(x) for x in next(f).split()]
            n_boxes = int(next(f))
            for it in range(n_boxes):
                id, l, rotx, w, roty, h, rotz, nn = [int(x) for x in next(f).split()] 
                if i == inst-1:
                    b = box(id, l, w, h, rotx, roty, rotz)
                    # Multiplicar nn para complejizar el problema
                    boxes[b]=nn*nbox;
                    id2box[id]=b
            if i == inst-1: 
                return L,W,H,boxes,id2box

### Ejemplo usando instancia BR

In [41]:
from bin import bin
import numpy as np

L,W,H,boxes,id2box = load_BRinstance(filename="BR8.txt",inst=4, nbox=1)

#Dimension del contenedor
L = int(L/2)
W = int(W/2)
H = int(H/2)

Vmax = L*W*H

In [42]:
def generate_candidate_solution(r_param = 1.0) -> list:
    solution_size = 0
    solution = np.array([])

    #Generating candidate Solution
    while(len(boxes) > 0):
        solution_size +=1 
        vol_c = 0
        c = dict()

        #Llenado del contenedor
        while(vol_c < r_param*Vmax and len(boxes) != 0):
            boxes_keys = list(boxes.keys())
            r =  random.randint(0, len(boxes_keys)-1)
            b = boxes_keys[r]
            n = min(boxes[b],8)    
            boxes[b] = boxes[b]-n

            # No quedan mas cajas del tipo b
            if(boxes[b] == 0):
                boxes.pop(b)

            #Agregar cantidad de cajas
            if(b not in c):
                c[b] = n
            else:
                c[b] = c[b]+n
            
            vol_box = b.vol
            vol_c = vol_c + vol_box*n

        #Evaluar contenedor cajas obtenidas 
        remaining, loaded, utilization = bsg_solve(L,W,H, c , id2box, time=5)
        container = bin(solution_size, loaded, utilization)
        #Se agrega la solucion
        solution = np.append(solution, container)
        
        # añadiendo las restantes para ser distribuidas nuevamente
        for key in remaining:
            if(key in boxes):
                boxes[key] += remaining[key]
            else:
                boxes[key] = remaining[key]

    print("Initial Solution: {}".format(len(solution)))
    return solution

In [43]:
def print_solution(solution:list):
    sol_size = 0
    for s in solution:
        
        vol = get_vol_by_boxes(boxes = s.boxes) /Vmax
        if vol != 0.0 :
            print("id: {} - vol: {}".format(s.id, vol))
            sol_size+=1
        
    print("Solution Size: {}".format(sol_size))

### Ejemplo creando instancia y resolviendo

In [44]:
# Retorna la probabilidad de llenar el
def calculate_prob(solution: list, metric:float) -> float:
    counter = 0
     
    for s in solution:
        v = float(get_vol_by_boxes(s.boxes)/Vmax)
        if(v >= metric and len(s.boxes) != 0 ):
            counter = counter + 1

    prob_calculate = (float)(counter/len(solution))
    return prob_calculate

In [45]:
def get_media_volumen(solution:list) -> float:
    ponderate = 0

    for s in solution:
        bin_boxes = s.boxes
        ponderate = ponderate + get_vol_by_boxes(bin_boxes)
    
    ponderate = ponderate/len(solution)
    return ponderate    

In [46]:
def get_vol_by_boxes(boxes: dict) -> float:
    vol = 0.0
    for box in boxes:
        vol += box.vol*boxes[box]
    return vol

In [47]:
def get_random_bin(s:list, b:bin = None) -> object:
    
    get_id = int(random.randint(0, len(s)-1))
    bin_selected = s[get_id]

    #Se retorna un bin distinto al seleccionado y que no este vacio
    while(b == bin_selected or len(bin_selected.boxes) == 0):
        get_id = int(random.randint(0, len(s)-1))
        bin_selected = s[get_id]
    
    return bin_selected


In [48]:
def pop_random_boxes_from_bin(last_bin: object, cant_boxes: int) -> dict:
    boxes_to_share = dict()
    boxes = last_bin.boxes

    for i in range(cant_boxes):
        total_keys = list(boxes.keys())
        
        #Selecciona una caja aleatoria
        get_id = int(random.randint(0, len(total_keys)-1))
        box = total_keys[get_id]
        boxes[box] = boxes[box]-1

        # Agrega la caja al conjunto
        if(box not in boxes_to_share):
            boxes_to_share[box] = 1
        else:
            boxes_to_share[box] = boxes_to_share[box]+1
        
        # Elimina la caja del conjunto si no quedan mas
        if(boxes[box] == 0):
            boxes.pop(box)
        
        if(len(boxes) == 0):
            break
    
    return boxes_to_share
    

In [49]:
def eval_bin(solution:list) -> float:
    quality = 0.0

    #cambiar a get_media_vo
    media = get_media_volumen(solution)

    for b in solution:
        bin_boxes = b.boxes
        quality = quality + pow((get_vol_by_boxes(bin_boxes)-media),2)

    return quality

In [50]:
def predict_vol(boxes:dict, box: object) -> float:
    vol_box = box.vol
    vol_bin = get_vol_by_boxes(boxes)

    total_vol = float((vol_box + vol_bin)/Vmax)
    
    return total_vol


In [51]:
def verify_solution(solution: list) -> list:
    final = np.array([])
    
    for s in solution:
        boxes = s.boxes
        remaining, loaded, utilization = bsg_solve(L,W,H, boxes, id2box, time=5)
        if(remaining != 0):
            return None
        container = bin(s.id, loaded, utilization)
        final = np.append(final, container)
    
    return final


In [52]:
def random_swap(solution:list) -> list:
    
    # Seleccionar bin 1 y 2 
    bin_1 = get_random_bin(s = solution) 
    bin_2 = get_random_bin(s = solution, b = bin_1)
    
    v1 = float(get_vol_by_boxes(bin_1.boxes)/Vmax)
    v2 = float(get_vol_by_boxes(bin_2.boxes)/Vmax)
        
    print("Initial ")
    print("Bin 1: {} ".format(v1), bin_1)
    print("Bin 2: {} ".format(v2), bin_2)

    # Seleccionar boxes del bin 1
    total_boxes = int(random.randint(1, 2))
    boxes_1 = pop_random_boxes_from_bin(last_bin=bin_1, cant_boxes=total_boxes)

    # Se agregan cajas de bin_1 en bin_2
    for box in boxes_1:
        if(box not in bin_2.boxes):
            bin_2.boxes[box] = boxes_1[box]
        else:
            bin_2.boxes[box] = bin_2.boxes[box] + boxes_1[box]
    
    p2 = calculate_prob(solution, bin_2.utilization)  
    # Seleccionar cajas desde el bin b_2
    while( p2 >= random.random() ):
        
        v1 = float(get_vol_by_boxes(bin_1.boxes)/Vmax)
        v2 = float(get_vol_by_boxes(bin_2.boxes)/Vmax)
        p1 = calculate_prob(solution, v1) # -> 0: No se puede llenar
        
        print("Before Swap")
        print("Bin 1: {} ".format(v1), bin_1)
        print("Bin 2: {} ".format(v2), bin_2)

        # Se espera solo una caja desde bin_2
        boxes_2 = pop_random_boxes_from_bin(last_bin=bin_2, cant_boxes= 1)
        box_2 = list(boxes_2.keys())[0] 

        if(predict_vol(bin_1.boxes, box_2) > 1):
            print("La caja escogida sobrepasa el máximo del bin 1")
            return solution

        # Se agrega la caja al conjunto del bin2 
        if(box_2 not in bin_1.boxes):  
            bin_1.boxes[box_2] = 1
        else:
            bin_1.boxes[box_2] = bin_1.boxes[box_2] + 1

        v1 = float(get_vol_by_boxes(bin_1.boxes)/Vmax)
        v2 = float(get_vol_by_boxes(bin_2.boxes)/Vmax)
        p2 = calculate_prob(solution, v2)

        print("After Swap")
        print("Bin 1: {} ".format(v1), bin_1)
        print("Bin 2: {} ".format(v2), bin_2)

        if(len(bin_2.boxes) == 0):
            print("Empty bin 2")
            break
          

        
    # retornar nueva solucion
    return solution

In [53]:
import copy as cp

def mclp(initial_solution: list, cant_test: int, r_param = 1.0) -> list: 
    
    best_solution = cp.deepcopy(initial_solution)
    best_eval = eval_bin(solution=initial_solution)
        
    for i in range(cant_test):
        print("--------------------- Swapping {} -------------------- ".format(i))
        current_solution = cp.deepcopy(best_solution)
        current_solution = random_swap(solution = current_solution)
        if(current_solution is not None):
            current_eval = eval_bin(solution=current_solution)
            print("Best {} - Eval {} ".format(best_eval, current_eval))
            if(current_eval > best_eval):   
                current_solution = verify_solution(current_solution)
                if(current_solution is not None):
                    print("Nueva solucion")
                    best_solution = cp.deepcopy(current_solution)
                    best_eval = current_eval
                else:
                    print("Solucion infactible")
        

    return best_solution



r_param = 1.5 
total_test = 50

#Creacion de solucion candidata
init_solution = generate_candidate_solution(r_param=r_param)
swapped_solution = mclp(initial_solution=init_solution, cant_test = total_test, r_param = r_param)


{"remaining" :[[26,2],[21,3],[27,4],[14,3]], "loaded" :[[11,8],[20,6],[2,6],[9,2],[21,2],[18,6]], "utilization" : 0.9341222}

{"remaining" :[[27,4],[17,2],[7,4],[24,1],[10,6]], "loaded" :[[17,6],[7,3],[15,3],[24,2],[5,2],[8,6],[3,3]], "utilization" : 0.9279045}

{"remaining" :[[30,5],[28,3]], "loaded" :[[30,3],[28,3],[4,8]], "utilization" : 0.87593134}

{"remaining" :[[28,3],[6,3]], "loaded" :[[13,3],[11,1],[22,5],[26,2],[6,1]], "utilization" : 0.81396134}

{"remaining" :[[12,1],[17,2],[19,4],[24,1],[23,2],[1,1]], "loaded" :[[25,3],[12,1],[7,4],[23,3],[1,3]], "utilization" : 0.88811452}

{"remaining" :[[28,3],[14,3]], "loaded" :[[23,2],[10,6],[30,5],[12,1]], "utilization" : 0.88247751}

{"remaining" :[[6,1],[17,1],[16,1],[28,3]], "loaded" :[[27,4],[6,2],[19,4],[17,1],[16,3],[24,1]], "utilization" : 0.85568596}

{"remaining" :[[28,3],[6,1],[17,1],[21,1],[1,1],[29,2]], "loaded" :[[21,2],[14,3],[29,6]], "utilization" : 0.80273038}

{"remaining" :[[28,2]], "loaded" :[[28,1],[16,1],[6,1],[2

In [54]:
print("Initial Solution")
print_solution(init_solution)
print("Swapped Solution")
print_solution(swapped_solution)

Initial Solution
id: 1 - vol: 0.9341222035584752
id: 2 - vol: 0.9279045010538479
id: 3 - vol: 0.8759313447526935
id: 4 - vol: 0.8139613446457038
id: 5 - vol: 0.888114521702847
id: 6 - vol: 0.8824775054297238
id: 7 - vol: 0.8556859640300855
id: 8 - vol: 0.8027303754266212
id: 9 - vol: 0.6607088598114843
id: 10 - vol: 0.3482298565269025
Solution Size: 10
Swapped Solution
id: 1 - vol: 0.9341222035584752
id: 2 - vol: 0.9279045010538479
id: 3 - vol: 0.8759313447526935
id: 4 - vol: 0.8139613446457038
id: 5 - vol: 0.888114521702847
id: 6 - vol: 0.8824775054297238
id: 7 - vol: 0.8556859640300855
id: 8 - vol: 0.8027303754266212
id: 9 - vol: 0.6607088598114843
id: 10 - vol: 0.3482298565269025
Solution Size: 10
